# Business Performance & Customer Insights Analysis — EDA

**PROJECT:** Business Performance & Customer Insights Analysis

**Tools:** Python (Pandas, NumPy, Matplotlib)

**Purpose:** End-to-end EDA on 10K+ sales & customer records to derive customer segments, contribution patterns, and operational performance insights.

## Section 0: Import Libraries


In [15]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import warnings

warnings.filterwarnings('ignore')

## Section 1: Load the Data


In [16]:
sales      = pd.read_csv('Sales_Transactions_Clean.csv', parse_dates=['transaction_date'])
customers  = pd.read_csv('Customer_Master_Clean.csv',    parse_dates=['registration_date'])
products   = pd.read_csv('Product_Master_Clean.csv')
dates      = pd.read_csv('Date_Dimension_Clean.csv',     parse_dates=['date'])


print(f"    Sales Transactions : {sales.shape[0]:,} rows × {sales.shape[1]} columns")
print(f"    Customer Master    : {customers.shape[0]:,} rows × {customers.shape[1]} columns")
print(f"    Product Master     : {products.shape[0]:,} rows × {products.shape[1]} columns")
print(f"    Date Dimension     : {dates.shape[0]:,} rows × {dates.shape[1]} columns")

    Sales Transactions : 12,501 rows × 16 columns
    Customer Master    : 2,500 rows × 13 columns
    Product Master     : 400 rows × 9 columns
    Date Dimension     : 182 rows × 8 columns


## Section 2: Data Overview & Quality Check

In [25]:
# Simple function to check basic dataset info
def check_data(df, name):
    print(f"\n--- {name} ---")
    print("Shape           :", df.shape)
    print("Total Nulls     :", df.isnull().sum().sum())
    print("Duplicate Rows  :", df.duplicated().sum())


# Check all datasets
check_data(sales, "Sales")
check_data(customers, "Customers")
check_data(products, "Products")
check_data(dates, "Dates")


# Null breakdown for Sales
print("\n--- Sales: Null Breakdown ---")
null_counts = sales.isnull().sum()

if null_counts.sum() == 0:
    print("No null values found. Data is clean.")
else:
    print(null_counts[null_counts > 0])


# Data types for Sales
print("\n--- Sales: Column Data Types ---")
print(sales.dtypes)


--- Sales ---
Shape           : (12501, 16)
Total Nulls     : 0
Duplicate Rows  : 0

--- Customers ---
Shape           : (2500, 13)
Total Nulls     : 0
Duplicate Rows  : 0

--- Products ---
Shape           : (400, 9)
Total Nulls     : 0
Duplicate Rows  : 0

--- Dates ---
Shape           : (182, 8)
Total Nulls     : 0
Duplicate Rows  : 0

--- Sales: Null Breakdown ---
No null values found. Data is clean.

--- Sales: Column Data Types ---
transaction_id                int64
transaction_date     datetime64[ns]
customer_id                   int64
product_id                   object
category                     object
quantity                      int64
unit_price                  float64
total_amount                float64
discount_applied            float64
payment_method               object
store_location               object
salesperson_id               object
profit                      float64
profit_margin_pct           float64
data_quality_flag            object
is_outlier        

## Section 3: Date Range & Time Coverage


In [24]:
# Get start and end date
start_date = sales['transaction_date'].min()
end_date = sales['transaction_date'].max()

# Calculate total months covered
total_months = ((end_date.year - start_date.year) * 12) + \
               (end_date.month - start_date.month) + 1

# Count unique sales days
unique_days = sales['transaction_date'].nunique()

# Print results
print("\n--- SALES PERIOD SUMMARY ---")
print("Sales Period :", start_date.date(), "to", end_date.date())
print("Total Months :", total_months)
print("Unique Dates :", unique_days)


--- SALES PERIOD SUMMARY ---
Sales Period : 2024-01-01 to 2024-06-30
Total Months : 6
Unique Dates : 182


## Section 4: KPI Summary — Core Business Metrics
These are the top-level numbers (Key Performance Indicators) that summarize the entire business health.

In [23]:

# Totals
total_revenue = sales['total_amount'].sum()
total_profit = sales['profit'].sum()
total_quantity = sales['quantity'].sum()

# Averages
avg_order_value = sales['total_amount'].mean()
avg_profit_margin = sales['profit_margin_pct'].mean()

# Counts
total_orders = sales['transaction_id'].nunique()
unique_customers = sales['customer_id'].nunique()


# Print Results
print("\n--- SALES SUMMARY ---")

print("Total Revenue        :", round(total_revenue, 2))
print("Total Profit         :", round(total_profit, 2))
print("Total Orders         :", total_orders)
print("Unique Customers     :", unique_customers)
print("Average Order Value  :", round(avg_order_value, 2))
print("Average Profit Margin:", round(avg_profit_margin, 2), "%")
print("Total Units Sold     :", total_quantity)


--- SALES SUMMARY ---
Total Revenue        : 355800575.38
Total Profit         : 161040776.75
Total Orders         : 12500
Unique Customers     : 2191
Average Order Value  : 28461.77
Average Profit Margin: 44.84 %
Total Units Sold     : 38237


## Section 5: Sales Performance Analysis
Here we break down the sales data by time, category, store, and payment method using a very simple step-by-step grouping method.

In [27]:
# 5a Monthly Performance
sales['year_month'] = sales['transaction_date'].dt.strftime('%Y-%m')

monthly = sales.groupby('year_month', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'profit': 'sum'
})

monthly = monthly.rename(columns={
    'total_amount': 'revenue',
    'transaction_id': 'orders'
})

print("\n[5a] Monthly Revenue Summary (first 6 rows):")
print(monthly.head(6).to_string(index=False))


# 5b Category Performance
category_perf = sales.groupby('category', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'profit': 'sum',
    'profit_margin_pct': 'mean',
    'quantity': 'sum'
})

category_perf = category_perf.rename(columns={
    'total_amount': 'revenue',
    'transaction_id': 'orders',
    'profit_margin_pct': 'avg_margin',
    'quantity': 'units'
})

category_perf = category_perf.sort_values(by='revenue', ascending=False)

total_rev = category_perf['revenue'].sum()
category_perf['revenue_share_%'] = (category_perf['revenue'] / total_rev * 100).round(2)

print("\n[5b] Category-wise Performance:")
print(category_perf.to_string(index=False))


# 5c Store / Channel Performance
store_perf = sales.groupby('store_location', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'profit': 'sum'
})

store_perf = store_perf.rename(columns={
    'total_amount': 'revenue',
    'transaction_id': 'orders'
})

store_perf = store_perf.sort_values(by='revenue', ascending=False)

print("\n[5c] Store / Channel Performance:")
print(store_perf.to_string(index=False))


# 5d Payment Method Distribution
payment_dist = sales.groupby('payment_method', as_index=False).agg({
    'transaction_id': 'count',
    'total_amount': 'sum'
})

payment_dist = payment_dist.rename(columns={
    'transaction_id': 'orders',
    'total_amount': 'revenue'
})

payment_dist = payment_dist.sort_values(by='orders', ascending=False)

print("\n[5d] Payment Method Distribution:")
print(payment_dist.to_string(index=False))


[5a] Monthly Revenue Summary (first 6 rows):
year_month     revenue  orders      profit
   2024-01 61607314.65    2098 27982098.65
   2024-02 56628234.85    1968 25729643.97
   2024-03 62480003.88    2189 28613274.48
   2024-04 61630013.80    2055 27380045.50
   2024-05 57214186.59    2086 26195262.97
   2024-06 56240821.61    2105 25140451.18

[5b] Category-wise Performance:
     category      revenue  orders       profit  avg_margin  units  revenue_share_%
  Electronics 285854390.11    2571 131253994.68   46.172085   3840            80.34
Home & Garden  39819886.07    2533  17026465.24   45.165191   8984            11.19
     Clothing  17463233.71    2757   7231101.27   43.197298   6908             4.91
        Books   6847246.76    2346   2935185.45   45.189557   5912             1.92
         Food   5815818.73    2294   2594030.11   44.578906  12593             1.63

[5c] Store / Channel Performance:
     store_location     revenue  orders      profit
   Store A (Mumbai) 93747796.

## Section 6: Customer Analytics
We join sales data with customer profiles to understand who is buying our products.

In [28]:
# Merge sales + customers
sales_cust = pd.merge(sales, customers, on='customer_id', how='left')


# 6a Customer Type Performance
cust_type = sales_cust.groupby('customer_type', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique'
})

cust_type = cust_type.rename(columns={
    'total_amount': 'revenue',
    'transaction_id': 'orders',
    'customer_id': 'unique_customers'
})

cust_type = cust_type.sort_values(by='revenue', ascending=False)
total_rev = cust_type['revenue'].sum()
cust_type['revenue_share_%'] = (cust_type['revenue'] / total_rev * 100).round(2)

print("\n[6a] Revenue by Customer Type:")
print(cust_type.to_string(index=False))


# 6b Gender-wise Revenue
gender_rev = sales_cust.groupby('gender', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count'
})

gender_rev = gender_rev.rename(columns={
    'total_amount': 'revenue',
    'transaction_id': 'orders'
})

print("\n[6b] Revenue by Gender:")
print(gender_rev.to_string(index=False))


# 6c Age Group Contribution
age_rev = sales_cust.groupby('age_group', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique'
})

age_rev = age_rev.rename(columns={
    'total_amount': 'revenue',
    'transaction_id': 'orders',
    'customer_id': 'customers'
})

age_rev = age_rev.sort_values(by='revenue', ascending=False)

print("\n[6c] Revenue by Age Group:")
print(age_rev.to_string(index=False))


# 6d Top 10 Customers
top10 = sales.groupby('customer_id', as_index=False).agg({
    'total_amount': 'sum',
    'transaction_id': 'count'
})

top10 = top10.rename(columns={
    'total_amount': 'total_revenue',
    'transaction_id': 'total_orders'
})

top10 = top10.sort_values(by='total_revenue', ascending=False).head(10)

details = customers[['customer_id', 'customer_name', 'customer_type', 'city']]
top10 = pd.merge(top10, details, on='customer_id', how='left')

cols = ['customer_id', 'customer_name', 'customer_type', 'city',
        'total_revenue', 'total_orders']

print("\n[6d] Top 10 Customers by Revenue:")
print(top10[cols].to_string(index=False))


# 6e Top 10 Cities
city_rev = sales_cust.groupby('city', as_index=False).agg({
    'total_amount': 'sum',
    'customer_id': 'nunique'
})

city_rev = city_rev.rename(columns={
    'total_amount': 'revenue',
    'customer_id': 'customers'
})

city_rev = city_rev.sort_values(by='revenue', ascending=False).head(10)

print("\n[6e] Top 10 Cities by Revenue:")
print(city_rev.to_string(index=False))


[6a] Revenue by Customer Type:
customer_type      revenue  orders  unique_customers  revenue_share_%
      Regular 180666045.12    6294              1107            50.78
          New 115226884.12    4091               726            32.39
          VIP  59907646.14    2116               358            16.84

[6b] Revenue by Gender:
gender      revenue  orders
Female 179229349.40    6307
  Male 176571225.98    6194

[6c] Revenue by Age Group:
age_group      revenue  orders  customers
    26-35 113211947.25    1947        355
    18-25 100138910.66    1737        303
    36-45  99412773.71    2275        388
    46-60  22289364.11    3238        560
      60+  20747579.65    3304        585

[6d] Top 10 Customers by Revenue:
 customer_id    customer_name customer_type      city  total_revenue  total_orders
        1766 Lakshit Nazareth           New   Kolkata     1706228.33            12
        2467       Anita Vala           VIP   Kolkata     1320944.71            13
         724   

## Section 7: Customer Retention Analysis
How often do our customers come back to buy again? We segment them based on purchase frequency.

In [29]:
print("\n" + "-" * 60)
print("SECTION 7 : Customer Retention Analysis")
print("-" * 60)


# Purchases per customer
customer_freq = sales.groupby('customer_id', as_index=False).agg({
    'transaction_id': 'count'
})

customer_freq = customer_freq.rename(columns={
    'transaction_id': 'num_purchases'
})


# Frequency segment
def get_segment(n):
    if n == 1:
        return '1 - One-time Buyer'
    elif n <= 3:
        return '2 - Occasional Buyer (2-3)'
    elif n <= 6:
        return '3 - Regular Buyer (4-6)'
    else:
        return '4 - Loyal Buyer (7+)'

customer_freq['freq_segment'] = customer_freq['num_purchases'].apply(get_segment)


# Segment summary
freq_summary = customer_freq.groupby('freq_segment', as_index=False).agg({
    'customer_id': 'count',
    'num_purchases': 'mean'
})

freq_summary = freq_summary.rename(columns={
    'customer_id': 'customers',
    'num_purchases': 'avg_purchases'
})

print("\nCustomer Purchase Frequency Segmentation:")
print(freq_summary.to_string(index=False))


# Retention rate
total_buyers = len(customer_freq)
repeat_buyers = len(customer_freq[customer_freq['num_purchases'] > 1])

retention_rate = (repeat_buyers / total_buyers) * 100

print("\nOverall Retention Rate (2+ purchases):",
      round(retention_rate, 2), "%")


------------------------------------------------------------
SECTION 7 : Customer Retention Analysis
------------------------------------------------------------

Customer Purchase Frequency Segmentation:
              freq_segment  customers  avg_purchases
        1 - One-time Buyer         80       1.000000
2 - Occasional Buyer (2-3)        400       2.597500
   3 - Regular Buyer (4-6)        904       4.977876
      4 - Loyal Buyer (7+)        807       8.527881

Overall Retention Rate (2+ purchases): 96.35 %


## Section 8: Product & Category Insights
Merging sales with product master data to find our top-selling specific items.

In [30]:
# Merge sales with product info
product_info = products[['product_id', 'product_name', 'sub_category', 'brand']]
sales_prod = pd.merge(sales, product_info, on='product_id', how='left')


# 8a Top 10 Products
top_products = sales_prod.groupby(
    ['product_id', 'product_name', 'category'],
    as_index=False
).agg({
    'total_amount': 'sum',
    'quantity': 'sum',
    'transaction_id': 'count'
})

top_products = top_products.rename(columns={
    'total_amount': 'revenue',
    'quantity': 'units',
    'transaction_id': 'orders'
})

top_products = top_products.sort_values(by='revenue', ascending=False).head(10)

print("\n[8a] Top 10 Products by Revenue:")
print(top_products[['product_name', 'category', 'revenue', 'units']]
      .to_string(index=False))


# 8b Sub-Category Performance (Top 15)
subcat = sales_prod.groupby(
    ['category', 'sub_category'],
    as_index=False
).agg({
    'total_amount': 'sum',
    'profit': 'sum',
    'quantity': 'sum'
})

subcat = subcat.rename(columns={
    'total_amount': 'revenue',
    'quantity': 'units'
})

subcat = subcat.sort_values(by='revenue', ascending=False).head(15)

print("\n[8b] Sub-Category Performance (Top 15):")
print(subcat.to_string(index=False))


[8a] Top 10 Products by Revenue:
        product_name    category    revenue  units
Veritatis Headphones Electronics 8026036.78     58
           Id Laptop Electronics 7668544.85     57
    Quam Smart Watch Electronics 7495813.75     67
 Possimus Smartphone Electronics 7049613.91     52
     Unde Smartphone Electronics 6973816.94     56
      Tenetur Tablet Electronics 6569275.30     54
    Alias Headphones Electronics 6471374.48     48
  Labore Smart Watch Electronics 6406915.12     51
          Eos Laptop Electronics 6336894.85     57
       Magnam Tablet Electronics 6100667.28     54

[8b] Sub-Category Performance (Top 15):
     category sub_category     revenue      profit  units
  Electronics   Headphones 66034701.03 30777352.71    891
  Electronics   Smartphone 62711908.23 31212235.78    666
  Electronics       Tablet 56825042.52 25962591.97    718
  Electronics  Smart Watch 53103888.70 22243827.89    904
  Electronics       Laptop 47178849.63 21057986.33    661
Home & Garden   

## Section 9: Discount Impact Analysis
Analyzing how offering discounts affects our total revenue and profit margin.

In [32]:
discount_impact = sales.groupby('discount_applied', as_index=False).agg({
    'transaction_id': 'count',
    'total_amount': 'sum',
    'profit_margin_pct': 'mean'
})

discount_impact = discount_impact.rename(columns={
    'transaction_id': 'orders',
    'total_amount': 'revenue',
    'profit_margin_pct': 'avg_margin'
})

discount_impact = discount_impact.sort_values(by='discount_applied')

print("\nRevenue & Margin by Discount %:")
print(discount_impact.to_string(index=False))


Revenue & Margin by Discount %:
 discount_applied  orders      revenue  avg_margin
              0.0    6217 188847983.21   44.791599
              5.0    2699  72713992.24   44.886002
             10.0    2780  72541168.73   44.824058
             15.0     710  19257716.53   45.219296
             20.0      48   1130386.89   44.359167
             25.0      47   1309327.78   43.042766


## Section 10: Outlier Summary
A quick look at unusually large transactions previously flagged during data cleaning.

In [33]:

outliers = sales[sales['is_outlier'] == 1]
clean_data = sales[sales['is_outlier'] == 0]

total_txn = len(sales)
outlier_count = len(outliers)
clean_count = len(clean_data)

outlier_pct = (outlier_count / total_txn) * 100

print("\nTotal transactions :", total_txn)
print("Flagged outliers   :", outlier_count, "(", round(outlier_pct, 2), "% )")
print("Clean transactions :", clean_count)

print("\nOutlier avg order value : INR",
      round(outliers['total_amount'].mean(), 2))

print("Clean avg order value   : INR",
      round(clean_data['total_amount'].mean(), 2))


Total transactions : 12501
Flagged outliers   : 1608 ( 12.86 % )
Clean transactions : 10893

Outlier avg order value : INR 153526.5
Clean avg order value   : INR 10000.0


## Section 11: Visualisations
We use standard, straightforward `pyplot` commands to generate and save charts. This avoids confusing "figure" and "axis" objects.

In [34]:
print("\n" + "-" * 60)
print("SECTION 11 : Generating Visualisations ...")
print("-" * 60)

BLUE = '#2E86AB'
GREEN = '#28A745'
ORANGE = '#E07B39'
RED = '#D62839'


# 1. Monthly Revenue Trend
plt.figure(figsize=(12, 5))

plt.plot(monthly['year_month'],
         monthly['revenue'] / 100000,
         marker='o',
         color=BLUE)

plt.title('Monthly Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Revenue (INR Lakhs)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.savefig('chart_01_monthly_revenue.png', dpi=120)
plt.close()
print("[Done] chart_01_monthly_revenue.png")


# 2. Category Revenue Share
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.pie(category_perf['revenue'],
        labels=category_perf['category'],
        autopct='%1.1f%%')
plt.title('Revenue Share by Category')

plt.subplot(1, 2, 2)
plt.bar(category_perf['category'],
        category_perf['avg_margin'],
        color=BLUE)
plt.title('Avg Profit Margin by Category')
plt.xticks(rotation=30)

plt.tight_layout()
plt.savefig('chart_02_category_analysis.png', dpi=120)
plt.close()
print("[Done] chart_02_category_analysis.png")


# 3. Customer Type Revenue
plt.figure(figsize=(8, 4))

plt.barh(cust_type['customer_type'],
         cust_type['revenue'] / 100000,
         color=GREEN)

plt.title('Revenue by Customer Type')
plt.xlabel('Revenue (INR Lakhs)')
plt.tight_layout()
plt.savefig('chart_03_customer_type.png', dpi=120)
plt.close()
print("[Done] chart_03_customer_type.png")


# 4. Age Group Revenue & Orders
plt.figure(figsize=(9, 5))

plt.bar(age_rev['age_group'],
        age_rev['revenue'] / 100000,
        color=BLUE)

ax2 = plt.twinx()
ax2.plot(age_rev['age_group'],
         age_rev['orders'],
         marker='o',
         color=ORANGE)

plt.title('Revenue & Orders by Age Group')
plt.tight_layout()
plt.savefig('chart_04_age_group.png', dpi=120)
plt.close()
print("[Done] chart_04_age_group.png")


# 5. Store Performance
plt.figure(figsize=(9, 5))

plt.bar(store_perf['store_location'],
        store_perf['revenue'] / 100000,
        color=BLUE)

plt.xticks(rotation=25)
plt.title('Revenue by Store / Channel')
plt.tight_layout()
plt.savefig('chart_05_store_performance.png', dpi=120)
plt.close()
print("[Done] chart_05_store_performance.png")


# 6. Retention Segments
plt.figure(figsize=(9, 5))

plt.bar(freq_summary['freq_segment'],
        freq_summary['customers'],
        color=ORANGE)

plt.xticks(rotation=20)
plt.title('Customer Frequency Segmentation')
plt.tight_layout()
plt.savefig('chart_06_retention_segments.png', dpi=120)
plt.close()
print("[Done] chart_06_retention_segments.png")


# 7. Top Cities
plt.figure(figsize=(9, 5))

plt.barh(city_rev['city'],
         city_rev['revenue'] / 100000,
         color=BLUE)

plt.title('Top 10 Cities by Revenue')
plt.tight_layout()
plt.savefig('chart_07_top_cities.png', dpi=120)
plt.close()
print("[Done] chart_07_top_cities.png")


# 8. Discount vs Margin
plt.figure(figsize=(8, 4))

x_labels = discount_impact['discount_applied'].astype(str) + '%'

plt.bar(x_labels,
        discount_impact['avg_margin'],
        color=ORANGE)

plt.axhline(discount_impact['avg_margin'].mean(),
            color=RED,
            linestyle='--')

plt.title('Avg Profit Margin by Discount')
plt.tight_layout()
plt.savefig('chart_08_discount_margin.png', dpi=120)
plt.close()
print("[Done] chart_08_discount_margin.png")


------------------------------------------------------------
SECTION 11 : Generating Visualisations ...
------------------------------------------------------------
[Done] chart_01_monthly_revenue.png
[Done] chart_02_category_analysis.png
[Done] chart_03_customer_type.png
[Done] chart_04_age_group.png
[Done] chart_05_store_performance.png
[Done] chart_06_retention_segments.png
[Done] chart_07_top_cities.png
[Done] chart_08_discount_margin.png


## Section 12: Key Insights Summary
This encapsulates the primary findings derived from the EDA, suitable for management reporting or portfolio highlights.

In [36]:
print("\n" + "=" * 60)
print("KEY INSIGHTS SUMMARY")
print("=" * 60)


top_cat = category_perf.iloc[0]
top_ctype = cust_type.iloc[0]
top_city = city_rev.iloc[0]
top_store = store_perf.iloc[0]

print("\nREVENUE & PROFIT")
print("-" * 60)
print("Total Revenue (Cr)      :", round(total_revenue / 10000000, 2))
print("Total Orders            :", total_orders)
print("Avg Profit Margin (%)   :", round(avg_profit_margin, 2))
print("Average Order Value     :", round(avg_order_value, 0))


print("\nCATEGORY INSIGHTS")
print("-" * 60)
print("Top Category            :", top_cat['category'])
print("Revenue Share (%)       :", round(top_cat['revenue_share_%'], 1))


print("\nCUSTOMER INSIGHTS")
print("-" * 60)
print("Top Customer Type       :", top_ctype['customer_type'])
print("Revenue Share (%)       :", round(top_ctype['revenue_share_%'], 1))
print("Retention Rate (%)      :", round(retention_rate, 1))
print("Top City                :", top_city['city'])
print("City Revenue (Lakhs)    :", round(top_city['revenue'] / 100000, 1))


print("\nOPERATIONAL INSIGHTS")
print("-" * 60)
print("Top Store / Channel     :", top_store['store_location'])
print("Store Revenue (Lakhs)   :", round(top_store['revenue'] / 100000, 1))
print("Outlier Transactions    :", outlier_count)
print("Outlier Percentage (%)  :", round(outlier_percentage, 1))




KEY INSIGHTS SUMMARY

REVENUE & PROFIT
------------------------------------------------------------
Total Revenue (Cr)      : 35.58
Total Orders            : 12500
Avg Profit Margin (%)   : 44.84
Average Order Value     : 28462.0

CATEGORY INSIGHTS
------------------------------------------------------------
Top Category            : Electronics
Revenue Share (%)       : 80.3

CUSTOMER INSIGHTS
------------------------------------------------------------
Top Customer Type       : Regular
Revenue Share (%)       : 50.8
Retention Rate (%)      : 96.3
Top City                : Pune
City Revenue (Lakhs)    : 490.7

OPERATIONAL INSIGHTS
------------------------------------------------------------
Top Store / Channel     : Store A (Mumbai)
Store Revenue (Lakhs)   : 937.5
Outlier Transactions    : 1608
Outlier Percentage (%)  : 12.9
